In [1]:
# demo storm coordinates
import pandas as pd
import numpy as np
import geojson
import matplotlib.pyplot as plt
%run -i storm_coords.py

#### Test dist_bearing function   
Now moved to `test_storm_coords.py`

#### Explore the geojson file a bit
Per Zach Cobell, these track data can be retrieved from using the METGET package, as follows:  
This information is based on the ATCF files posted by NHC, which is converted to JSON.   
We can also get you the raw ATCF as well. The metget software is pip installable via `pip install metget`.  
You only need to let the software know where the endpoint is by running:  
`export METGET_ENDPOINT=https://api.metget.org`    
`export METGET_API_KEY=0`  
(There is no api key necessary for this function, so setting it to zero is fine. If you want the native ATCF data, we can get you an API key)  

Then:
`metget track --storm 10 --year 2023 --type besttrack > idalia_track.json`

In [2]:
with open('/vortexfs1/home/csherwood/proj/NOPP_Idalia/idalia_track.json') as f:
    gj = geojson.load(f)
features = gj['features']
print(features[0])
print(features[0]['geometry'])
print(features[0]['geometry']['coordinates'][0])
print(features[0]['properties'])
print(features[0]['properties']['time_utc'])

print(len(features))

{"geometry": {"coordinates": [-84.0, 16.0], "type": "Point"}, "properties": {"max_wind_speed_mph": 17.26, "minimum_sea_level_pressure_mb": 0.0, "radius_to_max_wind_nmi": 0.0, "storm_class": "DB", "time_utc": "2023-08-24T18:00:00"}, "type": "Feature"}
{"coordinates": [-84.0, 16.0], "type": "Point"}
-84.0
{'time_utc': '2023-08-24T18:00:00', 'max_wind_speed_mph': 17.26, 'minimum_sea_level_pressure_mb': 0.0, 'radius_to_max_wind_nmi': 0.0, 'storm_class': 'DB'}
2023-08-24T18:00:00
65


#### Convert geojson into lists, convert to metric

In [3]:
mph2ms = 0.44704
nmi2km = 1.852
time_utc = []
lat = []
lon = []
max_wind_speed_ms = []
radius_to_max_wind_km = []
for feature in features:
    lon.append( feature['geometry']['coordinates'][0])
    lat.append( feature['geometry']['coordinates'][1])
    time_utc.append( feature['properties']['time_utc'])
    max_wind_speed_ms.append( feature['properties']['max_wind_speed_mph']*mph2ms )
    radius_to_max_wind_km.append( feature['properties']['radius_to_max_wind_nmi']*nmi2km )

# convert time stamps to datetime64
dt = pd.to_datetime(time_utc)

#### Convert to a dataframe and drop duplicates

In [4]:
# dfo = pd.DataFrame( {'time_utc': time_utc}, index = dt)
dfo = pd.DataFrame({'time_utc': time_utc,'lat': lat, 'lon': lon,
                    'max_wind_speed_ms': max_wind_speed_ms, 
                    'radius_to_max_wind_km': radius_to_max_wind_km}).drop_duplicates()

#### Compute distance and bearing between unique points, save dataframe to .csv

In [5]:
lon = np.array( dfo['lon'].values)
lat = np.array( dfo['lat'].values)

dist = np.ones_like( lon )
brg = np.ones_like( dist )
for i in range(len( lon )-1):
    dist[i],brg[i] = dist_bearing(lon[i], lat[i], lon[i+1], lat[i+1])

# make all the lists the same length
dist[i+1]=dist[i]
brg[i+1]=brg[i]

# add to the dataframe
dfo['distance_to_next_km']=dist
dfo['bearing_to_next_geo']=brg
dfo.to_csv('idalia_track_dist_brg.csv', index = False)